In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import json
import time

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm_notebook

In [5]:
X_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_train.csv', index_col=[0])
X_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_test.csv', index_col=[0])
y_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_train.csv', index_col=[0])
y_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_test.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

In [7]:
X_train_split = X_train['댓글'].apply(list).tolist()
X_test_split = X_test['댓글'].apply(list).tolist()

In [8]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)

In [ ]:
count_dict = tokenizer.word_counts

In [9]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

In [10]:
y_train.value_counts()

악플여부
0       54883
1       22541
dtype: int64

In [11]:
from imblearn.over_sampling import SMOTE

In [12]:
start = time.time()

smote = SMOTE(random_state=123)
train_over, y_train_over = smote.fit_sample(train, y_train)

print("걸린시간 :", time.time() - start)

걸린시간 : 228.86053800582886


In [13]:
pd.Series(y_train_over).value_counts()

1    54883
0    54883
dtype: int64

In [14]:
import tensorflow.keras.backend as K
# Mish 활성화 함수
def mish(x):
    return x * K.tanh(K.softplus(x))

In [15]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 32
comment_len = 400

In [16]:
def get_model() :

    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Flatten, MaxPool1D


    model = Sequential()
    model.add(Embedding(vocab_size+1, embedding_dim, input_length=comment_len))

    model.add(Dropout(0.2))
    model.add(Conv1D(32, 3, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.2))

    model.add(Dense(32))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

    return model


In [17]:
model = get_model()

In [18]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('1D_CNN_best.h5', monitor='val_loss', mode='min', save_best_only=True)

In [19]:
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 50,
                    batch_size       = 16, 
                    validation_data = (test, y_test))

Epoch 1/50
6861/6861 [==============================] - 35s 5ms/step - loss: 0.3973 - accuracy: 0.8236 - val_loss: 0.3401 - val_accuracy: 0.8791
Epoch 2/50
6861/6861 [==============================] - 35s 5ms/step - loss: 0.3458 - accuracy: 0.8571 - val_loss: 0.3414 - val_accuracy: 0.8701
Epoch 3/50
6861/6861 [==============================] - 35s 5ms/step - loss: 0.3316 - accuracy: 0.8659 - val_loss: 0.3069 - val_accuracy: 0.8928
Epoch 4/50
6861/6861 [==============================] - 35s 5ms/step - loss: 0.3250 - accuracy: 0.8690 - val_loss: 0.3135 - val_accuracy: 0.8883
Epoch 5/50
6861/6861 [==============================] - 36s 5ms/step - loss: 0.3201 - accuracy: 0.8718 - val_loss: 0.2971 - val_accuracy: 0.8961
Epoch 6/50
6861/6861 [==============================] - 35s 5ms/step - loss: 0.3160 - accuracy: 0.8747 - val_loss: 0.2909 - val_accuracy: 0.8932
Epoch 7/50
6861/6861 [==============================] - 34s 5ms/step - loss: 0.3156 - accuracy: 0.8748 - val_loss: 0.3171 - val_ac

In [20]:
# 저장한 LSTM 모델을 불러오고 예측해보자


loaded_model = load_model('1D_CNN_best.h5')
y_pred = loaded_model.predict(test)

y_class = y_pred > 0.5

In [21]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix

print('recall : ', recall_score(y_class, y_test))
print('precision : ', precision_score(y_class, y_test))
print('f1_score : ', f1_score(y_class, y_test))
print('roc_auc_score : ', roc_auc_score(y_test, y_pred))
print(confusion_matrix(y_class, y_test))

recall :  0.8771855908995155
precision :  0.7372521246458924
f1_score :  0.8011544011544013
roc_auc_score :  0.9230719926062074
[[13125  1484]
 [  583  4164]]


In [22]:
def model_test(comment):
    loaded_model = load_model('1D_CNN_best.h5')
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)

    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)

    pred = loaded_model.predict(comment_pad)

    if pred < 0.5 :
        result = '악플이 아닙니다'
    else :
        result = '악플입니다'
    
    print(result)
    print('라벨확률 : ', pred)
    
    return result, pred

In [186]:
comment = ''''''

result, prob = model_test(comment)

악플이 아닙니다
라벨확률 :  [[0.29582623]]
